In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import matplotlib.patches as patches
import matplotlib
import tifffile
import os
import numpy as np
import sys
import dxchange
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.ndimage as ndimage
import spin_average as sa
%matplotlib inline

In [ ]:
def halfbit(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    r=r.astype(np.int32)
    nr = np.bincount(r.ravel())
    return (0.2071+1.9102/np.sqrt(nr))/(1.2071+0.9102/np.sqrt(nr)) 

def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    r=r.astype(np.int32)
    

    tbinre = np.bincount(r.ravel(), data.real.ravel())
    tbinim = np.bincount(r.ravel(), data.imag.ravel())
    
    nr = np.bincount(r.ravel())
    radialprofile = (tbinre+1j*tbinim) / np.sqrt(nr)
    
    return radialprofile

In [ ]:
from scipy.signal import savgol_filter

def smooth_savgol(data, window_size, poly_order):
    """
    Smooths data using a Savitzky-Golay filter.

    Args:
      data: A 1D numpy array of data values.
      window_size: The length of the filter window (must be odd).
      poly_order: The order of the polynomial used to fit the data points.

    Returns:
      A 1D numpy array of smoothed data values.
    """
    if window_size % 2 == 0 or window_size < 1:
      raise ValueError("Window size must be an odd positive integer")
    if poly_order >= window_size:
       raise ValueError("poly_order must be less than window_size")
    data = np.pad(data,(32,32),'edge')
    print(data.shape)
    data = savgol_filter(data, window_size, poly_order)
    data=data[32:-32]
    return data

In [ ]:
a = np.hamming(512)
b = np.hamming(512)
ham = np.outer(a,b)
plt.imshow(ham)
plt.colorbar()

In [ ]:

for wsize in [3072]:#512,1024,2048,2048+512,2048+1024]:
    fig,ax = plt.subplots(1,1,figsize=(10,5))    
    iter = 384
    # wsize = 3096
    f1 = tifffile.imread(f'/data/vnikitin/ESRF/ID16A/20240924_rec0224/Chip_nfp/s22/rec_prb_angle/{iter:04}.tiff')
    f2 = tifffile.imread(f'/data/vnikitin/ESRF/ID16A/20240924_rec0224/Chip_nfp/s33/rec_prb_angle/{iter:04}.tiff')
    plt.rcParams['axes.labelsize'] = 60
    plt.rcParams['axes.titlesize'] = 32

    f1 = f1[f1.shape[0]//2-wsize:f1.shape[0]//2+wsize,f1.shape[1]//2-wsize:f1.shape[1]//2+wsize]
    f2 = f2[f2.shape[0]//2-wsize:f2.shape[0]//2+wsize,f2.shape[1]//2-wsize:f2.shape[1]//2+wsize]

    a = np.hamming(2*wsize)
    ham = np.outer(a,a)
    f1*=ham
    f2*=ham

    ff1 = np.fft.fftshift(np.fft.fft2(f1))
    ff2 = np.fft.fftshift(np.fft.fft2(f2))

    # plt.imshow(np.log(np.abs(ff1)))
    # plt.colorbar()
    # plt.show()
    
    # plt.imshow((np.abs(fff1-ff1)))
    # plt.colorbar()
    # plt.show()

    # plt.show()
    print(ff1.shape)
    
    frc1 = radial_profile(ff1*np.conj(ff2),np.array(ff1.shape)//2)/\
        np.sqrt(radial_profile(np.abs(ff1)**2,np.array(ff1.shape)//2)*radial_profile(np.abs(ff2)**2,np.array(ff1.shape)//2))
    a = np.hamming(2*len(frc1))
    print(a.shape)
    a = a[len(a)//2:]
    frc1*=a
    print(frc1.shape,a.shape)
    frc1=np.abs(frc1)[:wsize]
    np.save('frc1.npy',frc1)

    ff1 = np.zeros([2*wsize,2*wsize],dtype='complex64')
    hbit = halfbit(ff1,np.array([2*wsize,2*wsize])//2)
    np.save('hbit.npy',hbit)

    frc1=np.load('frc1.npy')
    frc1c=smooth_savgol(frc1, 51, 3)
    # frc1c[-wsize//16:]=frc1c[-wsize//16:]
    hbit=np.load('hbit.npy')
    ax.plot(frc1[:].real,linewidth=1.5, label=f'FRC')
    ax.plot(frc1c[:].real,linewidth=1.5, label=f'FRC smooth')
    
    ax.plot(hbit[:wsize],linewidth=1.5,label='1/2-bit')

    ax.grid()
    ax.set_ylim([0,1])
    lg = plt.legend(loc="upper right",fontsize=16)
    lg.get_title().set_fontsize(16)
    plt.ylabel('FRC',rotation=90, fontsize = 16)
    ax2 = ax.secondary_xaxis('top')
    ax.set_xlabel('Spatial/Nyquist frequency', fontsize=16)
    ax2.set_xlabel('Spatial resolution (nm)', fontsize=16)
    ax.set_xticks(np.linspace(0,wsize,11),np.int32(100*np.linspace(0,1,11))/100,fontsize=10)
    ax2.set_xticks(np.linspace(0,wsize,11),(np.int32(1667/np.linspace(1,10,11))/100),fontsize=10)
    # plt.show()
    plt.savefig(f'{iter}_{wsize}.png',dpi=300)

In [ ]:
iter = 1536
path = '/data/vnikitin/ESRF/ID16A/20240924_rec0224/Chip_nfp/'
names = ['s22','s33'
         ]
for name in names:#os.listdir(path):
    print(name)
    try:
        psi = tifffile.imread(f'{path}/{name}/rec_psi_abs/{iter:04}.tiff')*np.exp(1j*tifffile.imread(f'{path}/{name}/rec_psi_angle/{iter:04}.tiff'))
        prb = tifffile.imread(f'{path}/{name}/rec_prb_abs/{iter:04}.tiff')*np.exp(1j*tifffile.imread(f'{path}/{name}/rec_prb_angle/{iter:04}.tiff'))

        fig, ax = plt.subplots(2,2,figsize=(16,16))
        im = ax[0,1].imshow(np.angle(psi),cmap='gray',vmin=-0.4,vmax=0.2)
        rect = patches.Rectangle((psi.shape[-1]//2-3072, psi.shape[-1]//2-3072), 6144, 6144, linewidth=1, edgecolor='r', facecolor='none')
        ax[0,1].add_patch(rect)

        # fig.colorbar(im, fraction=0.046, pad=0.02)

        im = ax[0,0].imshow(np.abs(psi),cmap='gray',vmin=0.8,vmax=1.1)
        rect = patches.Rectangle((psi.shape[-1]//2-3072, psi.shape[-1]//2-3072), 6144, 6144, linewidth=1, edgecolor='r', facecolor='none')
        ax[0,0].add_patch(rect)
        # fig.colorbar(im, fraction=0.046, pad=0.02)

        im = ax[1,1].imshow(np.angle(prb),cmap='gray',vmin=-1.8,vmax=1.8)
        rect = patches.Rectangle((prb.shape[-1]//2-3072, prb.shape[-1]//2-3072), 6144, 6144, linewidth=1, edgecolor='r', facecolor='none')
        ax[1,1].add_patch(rect)
        # fig.colorbar(im, fraction=0.046, pad=0.02)

        im = ax[1,0].imshow(np.abs(prb),cmap='gray',vmin=0.4,vmax=1.9)
        rect = patches.Rectangle((prb.shape[-1]//2-3072, prb.shape[-1]//2-3072), 6144, 6144, linewidth=1, edgecolor='r', facecolor='none')
        ax[1,0].add_patch(rect)
        # fig.colorbar(im, fraction=0.046, pad=0.02)
        plt.suptitle(name)
        plt.savefig(f'{name}.png',dpi=300,bbox_inches='tight')

        ax[0,0].set_xlim([psi.shape[-1]//2+512*3+256*3,psi.shape[-1]])
        ax[0,0].set_ylim([psi.shape[-1]//2-384*3,psi.shape[-1]//2+128*3])
        ax[0,1].set_xlim([psi.shape[-1]//2+512*3+256*3,psi.shape[-1]])
        ax[0,1].set_ylim([psi.shape[-1]//2-384*3,psi.shape[-1]//2+128*3])
        ax[1,0].set_xlim([prb.shape[-1]//2+512*3+256*3,prb.shape[-1]])
        ax[1,0].set_ylim([prb.shape[-1]//2-384*3,prb.shape[-1]//2+128*3])
        ax[1,1].set_xlim([prb.shape[-1]//2+512*3+256*3,prb.shape[-1]])
        ax[1,1].set_ylim([prb.shape[-1]//2-38*34,prb.shape[-1]//2+128*3])

        plt.savefig(f'{name}zoom.png',dpi=300,bbox_inches='tight')
        plt.show()
    except:
        pass

